# Automatic Number Plate Recognition

In [ ]:
import os

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

p=r'C:\tensorflow'

paths = {
    'WORKSPACE_PATH': os.path.join(p,'Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join(p,'Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join(p,'Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join(p,'Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(p,'Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join(p,'Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join(p,'Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(p,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(p,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(p,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(p,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join(p,'Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join(p,'Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
%cd C:\tensorflow

# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
!pip list

In [ ]:
pip install numpy
pip install google-api-python-client
pip install protobuf==3.20.*
pip install wrapt
pip install opt_einsum
pip install gast
pip install astunparse
pip install termcolor
pip install flatbuffers
pip install matplotlib
pip install pyyaml
pip install gin-config
!pip install tensorflow-addons



In [ ]:
pip install pyyaml


In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
%cd C:\tensorflow\Tensorflow\models\research\build\lib
import object_detection

In [ ]:
!pip list

Downloading the transfer learning model

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [ ]:
files['LABELMAP']

# Creating label map, and from it we assign labels

In [ ]:
%cd C:\Users\Lenovo\Desktop\anpr\c
labels = [{'name':'licence', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Converts annotations in your images into a file format that can be used

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}


In [ ]:
pip install pytz

# These line of code creates a record file so that it can be loaded

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 


# Copy the model configuration file to training folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}


# Updating the config file for transfer learning according to our requirements

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
files['PIPELINE_CONFIG']

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# TRAINING THE MODEL

In [ ]:
paths['APIMODEL_PATH']

In [ ]:
TRAINING_SCRIPT=os.path.join(paths['APIMODEL_PATH'],'research','object_detection','model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
print(command)

================================================================================================

The number of steps define for how long the model will train

In [ ]:
!(command)

# Prevent GPU complete consumption


GPU shouldn't be consumed fully because easyOCR uses pytorch which is also graphic intensive alongside tensorflow

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try: 
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
    except RunTimeError as e:
        print(e)

# Loading the trained model from checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util, visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detection on a single frame(image)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
category_index=label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH=os.path.join(paths['IMAGE_PATH'],'test','Cars0.png')

The purpose of a TensorFlow tensor object is to represent multi-dimensional arrays or tensors of data, which are the basic building blocks of any machine learning model or data processing pipeline built with TensorFlow.

A tensor is a generalization of a matrix, which can have an arbitrary number of dimensions. For example, a 1D tensor represents a vector, a 2D tensor represents a matrix, a 3D tensor represents a cube, and so on. A tensor can hold data of any data type, such as integers, floats, or strings, and can be used to represent a variety of data, including images, audio signals, text, and more.

In [ ]:
img=cv2.imread(IMAGE_PATH) #reads an image
image_np=np.array(img) #converts the image into an array

input_tensor=tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32) #creates a TensorFlow tensor object from a NumPy array
detections=detect_fn(input_tensor)

num_detections=int(detections.pop('num_detections'))
detections={key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes']=detections['detection_classes'].astype(np.int64)

label_id_offset=1
image_np_with_detections=image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8, #if the accuracy is above 80%, then point out
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
detections.keys()

# REAL TIME DETECTION USING WEBCAM

In [ ]:
cap=cv2.VideoCapture(0)
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame=cap.read()
    image_np=np.array(frame)
    input_tensor=tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32) #creates a TensorFlow tensor object from a NumPy array
    detections=detect_fn(input_tensor)

    num_detections=int(detections.pop('num_detections'))
    detections={key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    label_id_offset=1
    image_np_with_detections=image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8, #if the accuracy is above 80%, then point out
            agnostic_mode=False)
    cv2.imshow('object detection', cv2.resize(image_np_with_detections,(800,600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


# APPLY OCR to DETECTION

In [ ]:
!pip install easyocr

Install pytorch for gpu acceleration

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
pip install chardet

In [ ]:
pip install --force-reinstall charset-normalizer==3.1.0

In [ ]:
import easyocr 

In [ ]:
detection_threshold=0.7 #70% accuracy

In [ ]:
image=image_np_with_detections #same image as above
scores=list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
boxes=detections['detection_boxes'][:len(scores)]
classes=detections['detection_classes'][:len(scores)]

In [ ]:
detections['detection_scores']

In [ ]:
detections['detection_scores'] > detection_threshold

In [ ]:
scores

In [ ]:
detections['detection_boxes']

In [ ]:
detections['detection_classes']

In [ ]:
boxes

In [ ]:
width=image.shape[1]
height=image.shape[0]

In [ ]:
for idx, box in enumerate(boxes):
    roi=box*[height,width,height,width]
    region=image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
    reader=easyocr.Reader(['en'])
    ocr_result=reader.readtext(region)
    print(ocr_result)
    plt.imshow(region)

In [ ]:
region_threshold=0.6

In [ ]:
def filter_text(region,ocr_result,region_threshold):
    rectangle_size=region.shape[0]*region.shape[1]
    plate=[]
    for result in ocr_result:
        length=np.sum(np.subtract(result[0][1],result[0][0]))
        height=np.sum(np.subtract(result[0][2],result[0][1]))
        
        if length*height/rectangle_size > region_threshold:
            plate.append(result[1])
        
#         print(length,height)
    return plate

In [ ]:
filter_text(region,ocr_result,region_threshold)

In [ ]:
def ocr_it(image, detections, detection_threshold, region_threshold):
    
    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]
    
    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)
        
        text = filter_text(region, ocr_result, region_threshold)
        
#         plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
#         plt.show()
        if len(text)!=0:
            print(text)
        return text, region

In [ ]:
text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)



# REAL TIME DETECTION USING WEBCAM

In [ ]:
import cv2

In [ ]:
pip install opencv-python --upgrade

In [ ]:
import cv2


In [ ]:
cap=cv2.VideoCapture(0)
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened():
    ret, frame=cap.read()
    image_np=np.array(frame)
    input_tensor=tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32) #creates a TensorFlow tensor object from a NumPy array
    detections=detect_fn(input_tensor)

    num_detections=int(detections.pop('num_detections'))
    detections={key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    label_id_offset=1
    image_np_with_detections=image_np.copy()
#     viz_utils.visualize_boxes_and_labels_on_image_array(
#             image_np_with_detections,
#             detections['detection_boxes'],
#             detections['detection_classes']+label_id_offset,
#             detections['detection_scores'],
#             category_index,
#             use_normalized_coordinates=True,
#             max_boxes_to_draw=5,
#             min_score_thresh=.8, #if the accuracy is above 80%, then point out
#             agnostic_mode=False)
    
    try:
        text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)
    except:
        pass
    
    cv2.imshow('object detection', cv2.resize(image_np_with_detections,(800,600)))

    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()